# Relationship prediction experiments

## Run logistic regression to predict romantic relationships, other

In [1]:
import pandas as pd

# Load data
data = pd.read_pickle('/usr2/mamille2/fanfiction-project/data/features/relationship_prediction_assertion_sample5400.pkl')
print(len(data))
print(data.columns)

pairings = data['pairing'].unique()

5400
Index(['fic_id', 'pairing', 'pairing_embedding_unigrams_assertions_local_5',
       'pairing_embedding_unigrams_assertions_local_10',
       'pairing_embedding_unigrams_assertions_local_25',
       'pairing_embedding_unigrams_assertions_local_50', 'relationship',
       'selected_relationships', 'is_romantic', 'is_canon', 'is_mm',
       'shape_10', 'pairing_embedding_embs_adapted_10',
       'pairing_embedding_unigrams_quotes', 'dist_canon_combined',
       'subtraction_canon_combined', 'dist_canon_bg', 'subtraction_canon_bg',
       'dist_canon_aligned', 'subtraction_canon_aligned',
       'pairing_embedding_embs_background_quotes',
       'pairing_embedding_embs_combined_quotes',
       'pairing_embedding_canon_embs_aligned',
       'pairing_embedding_embs_aligned', 'pairing_embedding_embs_combined',
       'pairing_embedding_canon_embs_combined',
       'pairing_embedding_embs_background',
       'pairing_embedding_canon_embs_background', 'dist_canon_background',
       'subtr

In [2]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

# train, test = train_test_split(data, test_size=0.1, random_state=9)
train_ids, test_ids = train_test_split(data['fic_id'].unique(), test_size=0.1, random_state=11)
# train_ids, test_ids = train_test_split(data['fic_id'].unique(), test_size=0.1, random_state=9)

train = data[data['fic_id'].isin(train_ids)]
test = data[data['fic_id'].isin(test_ids)]
print(len(train))
print(len(train)/ len(data))
print(len(test))
print(train.columns)
print(sum(train['is_romantic']/len(train)))
print(sum(train['is_canon']/len(train)))
print(sum(train['is_mm']/len(train)))

4866
0.9011111111111111
534
Index(['fic_id', 'pairing', 'pairing_embedding_unigrams_assertions_local_5',
       'pairing_embedding_unigrams_assertions_local_10',
       'pairing_embedding_unigrams_assertions_local_25',
       'pairing_embedding_unigrams_assertions_local_50', 'relationship',
       'selected_relationships', 'is_romantic', 'is_canon', 'is_mm',
       'shape_10', 'pairing_embedding_embs_adapted_10',
       'pairing_embedding_unigrams_quotes', 'dist_canon_combined',
       'subtraction_canon_combined', 'dist_canon_bg', 'subtraction_canon_bg',
       'dist_canon_aligned', 'subtraction_canon_aligned',
       'pairing_embedding_embs_background_quotes',
       'pairing_embedding_embs_combined_quotes',
       'pairing_embedding_canon_embs_aligned',
       'pairing_embedding_embs_aligned', 'pairing_embedding_embs_combined',
       'pairing_embedding_canon_embs_combined',
       'pairing_embedding_embs_background',
       'pairing_embedding_canon_embs_background', 'dist_canon_bac

In [4]:
import numpy as np
import scipy.sparse

X_train, y_train = {}, {}
X_test, y_test = {}, {}

add_features = True

# for feature_set in ['unigrams', 'embs']:
# for feature_set in ['unigrams_assertions_local_10', 'embs_adapted_10', 'embs_background_10']:
# for feature_set in ['unigrams_quotes', 'unigrams_assertions_local_10']:
# for feature_set in pairing_char_embs_fpath:
# for feature_set in ['embs_combined', 'dist_canon_combined']:
# for feature_set in ['embs_combined', 'subtraction_canon_combined']:
# for feature_set in ['embs_background', 'dist_canon_background']:
# for feature_set in ['embs_background', 'subtraction_canon_background']:
# for feature_set in ['embs_combined', 'dist_canon_aligned']:
# for feature_set in ['embs_combined', 'subtraction_canon_aligned']:
# for feature_set in ['embs_combined', 'pairing_embedding_embs_combined_quotes']:
# for feature_set in ['embs_background', 'embs_background_quotes', 'subtraction_canon_background']:
for feature_set in ['embs_combined', 'embs_combined_quotes', 'subtraction_canon_combined']: # best results
# for feature_set in ['embs_combined', 'embs_combined_quotes', 'dist_canon_combined']:
# for feature_set in ['embs_combined', 'embs_combined_quotes']:
# for feature_set in ['embs_combined']:
# for feature_set in ['embs_combined', 'embs_combined_quotes', 'subtraction_canon_aligned']:
# for feature_set in ['embs_combined', 'embs_combined_quotes', 'dist_canon_aligned']:

    if feature_set.startswith('embs'):
        X_train[feature_set] = np.vstack(train[f'pairing_embedding_{feature_set}'].tolist())
        X_test[feature_set] = np.vstack(test[f'pairing_embedding_{feature_set}'].tolist())
        
    elif 'canon' in feature_set:
        X_train[feature_set] = np.vstack(train[feature_set].tolist())
        X_test[feature_set] = np.vstack(test[feature_set].tolist())
        
    elif feature_set.startswith('unigrams'):
        X_train[feature_set] = scipy.sparse.vstack(train[f'pairing_embedding_{feature_set}'].tolist())
        X_test[feature_set] = scipy.sparse.vstack(test[f'pairing_embedding_{feature_set}'].tolist())
        
    elif feature_set == 'unigrams+embs':
        unigram_features = scipy.sparse.vstack(train[f'pairing_embedding_unigrams'].tolist())
        emb_features = np.array(train[f'pairing_embedding_embs'].tolist())
        X_train[feature_set] = scipy.sparse.hstack([unigram_features, emb_features])
        
        unigram_features = scipy.sparse.vstack(test[f'pairing_embedding_unigrams'].tolist())
        emb_features = np.array(test[f'pairing_embedding_embs'].tolist())
        X_test[feature_set] = scipy.sparse.hstack([unigram_features, emb_features])
        
    if add_features:
        if 'unigrams' in ' '.join(feature_set):
            X_train['all'] = scipy.sparse.hstack(list(X_train.values()))
            X_test['all'] = scipy.sparse.hstack(list(X_test.values()))
        
        else:
            X_train['all'] = np.hstack(list(X_train.values()))
            X_test['all'] = np.hstack(list(X_test.values()))
        
y_train['is_romantic'] = train['is_romantic']
y_train['is_canon'] = train['is_canon']
y_train['is_mm'] = train['is_mm']

y_test['is_romantic'] = test['is_romantic']
y_test['is_canon'] = test['is_canon']
y_test['is_mm'] = test['is_mm']

X_train.keys()

# Per-instance score
results = {}
clf = {}

# for feature_set in ['unigrams_assertions_local_10']:
# for feature_set in ['embs_adapted_10']:
# for feature_set in ['embs_aligned_10']:
# for feature_set in ['embs_background_10']:
# for feature_set in ['unigrams_assertions_local_10', 'embs_adapted_10', 'embs_background_10']:
# for feature_set in ['unigrams_quotes']:
# for feature_set in ['all']:
# for feature_set in pairing_char_embs_fpath:
for feature_set in ['all']:
    results[f'{feature_set}_lr'] = {}
    clf[f'{feature_set}_lr'] = {}
    for key in y_test:

        clf[f'{feature_set}_lr'][key] = LogisticRegression(solver='liblinear', max_iter=1000)
        clf[f'{feature_set}_lr'][key].fit(X_train[feature_set], y_train[key])
        results[f'{feature_set}_lr'][key] = clf[f'{feature_set}_lr'][key].score(X_test[feature_set], y_test[key])
        print(f'{feature_set}\t{key}: {results[f"{feature_set}_lr"][key]}')
        
    print()

# Group instance IDs
test_groups = test.reset_index().groupby('fic_id').groups
len(test_groups)

# Per-fic score
perfic_results = {}
predictions = {}

# for feature_set in ['unigrams_assertions_local_10']:
# for feature_set in ['embs_adapted_10']:
# for feature_set in ['embs_background_10']:
# for feature_set in ['unigrams_assertions_local_10', 'embs_adapted_10', 'embs_background_10']:
# for feature_set in ['unigrams_quotes']:
for feature_set in ['all']:
# for feature_set in pairing_char_embs_fpath:
    perfic_results[f'{feature_set}_lr'] = {}
    for key in y_test:

        predictions[key] = clf[f'{feature_set}_lr'][key].predict(X_test[feature_set])
        
        avg_acc = []
        for fic_id, indices in test_groups.items():
            avg_acc.append(np.mean([predictions[key][i]==y_test[key].tolist()[i] for i in indices]))
        
        perfic_results[f'{feature_set}_lr'][key] = np.mean(avg_acc)
        print(f'{feature_set}\t{key}: {perfic_results[f"{feature_set}_lr"][key]}')
        
    print()

all	is_romantic: 0.795880149812734
all	is_canon: 0.6910112359550562
all	is_mm: 0.900749063670412

all	is_romantic: 0.7788072417465388
all	is_canon: 0.6852502662406815
all	is_mm: 0.8882321618743344



In [11]:
# Group instance IDs
train_groups = train.reset_index().groupby('fic_id').groups
len(train_groups)

# Per-fic score
train_perfic_results = {}
train_predictions = {}

# for feature_set in ['unigrams_assertions_local_10']:
# for feature_set in ['embs_adapted_10']:
# for feature_set in ['embs_background_10']:
# for feature_set in ['unigrams_assertions_local_10', 'embs_adapted_10', 'embs_background_10']:
# for feature_set in ['unigrams_quotes']:
for feature_set in ['all']:
# for feature_set in pairing_char_embs_fpath:
    train_perfic_results[f'{feature_set}_lr'] = {}
    for key in y_train:

        train_predictions[key] = clf[f'{feature_set}_lr'][key].predict(X_train[feature_set])
        
        train_avg_acc = []
        for fic_id, indices in train_groups.items():
            train_avg_acc.append(np.mean([train_predictions[key][i]==y_train[key].tolist()[i] for i in indices]))
        
        train_perfic_results[f'{feature_set}_lr'][key] = np.mean(train_avg_acc)
        print(f'{feature_set}\t{key}: {train_perfic_results[f"{feature_set}_lr"][key]}')
        
    print()

all	is_romantic: 0.8211190053285967
all	is_canon: 0.7396092362344583
all	is_mm: 0.9221432800473652



In [ ]:
# Join training predictions with fic info, save out

In [12]:
X_train['all'].shape

(4866, 1000)

In [13]:
len(train_predictions['is_canon'])

4866

In [14]:
len(train)

4866

In [15]:
train.columns

Index(['fic_id', 'pairing', 'pairing_embedding_unigrams_assertions_local_5',
       'pairing_embedding_unigrams_assertions_local_10',
       'pairing_embedding_unigrams_assertions_local_25',
       'pairing_embedding_unigrams_assertions_local_50', 'relationship',
       'selected_relationships', 'is_romantic', 'is_canon', 'is_mm',
       'shape_10', 'pairing_embedding_embs_adapted_10',
       'pairing_embedding_unigrams_quotes', 'dist_canon_combined',
       'subtraction_canon_combined', 'dist_canon_bg', 'subtraction_canon_bg',
       'dist_canon_aligned', 'subtraction_canon_aligned',
       'pairing_embedding_embs_background_quotes',
       'pairing_embedding_embs_combined_quotes',
       'pairing_embedding_canon_embs_aligned',
       'pairing_embedding_embs_aligned', 'pairing_embedding_embs_combined',
       'pairing_embedding_canon_embs_combined',
       'pairing_embedding_embs_background',
       'pairing_embedding_canon_embs_background', 'dist_canon_background',
       'subtractio

In [17]:
train.loc[:,'best_pred_canon'] = train_predictions['is_canon']

/usr0/home/mamille2/anaconda/lib/python3.7/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [19]:
# Save out
train.to_pickle('/usr2/mamille2/fanfiction-project/output/emnlp2020_predictions/relationship_prediction_sample5400_train.pkl')

# Look into correct, incorrect predictions on training set

In [21]:
true_negatives = train[(train['best_pred_canon']==False) & (train['is_canon']==False)]
true_positives = train[(train['best_pred_canon']==True) & (train['is_canon']==True)]
false_positives = train[(train['best_pred_canon']==True) & (train['is_canon']==False)]
false_negatives = train[(train['best_pred_canon']==False) & (train['is_canon']==True)]

print(f'Number of true negatives: {len(true_negatives)}')
print(f'Number of true positives: {len(true_positives)}')
print(f'Number of false positives: {len(false_positives)}')
print(f'Number of false negatives: {len(false_negatives)}')

Number of true negatives: 1432
Number of true positives: 2207
Number of false positives: 703
Number of false negatives: 524
